<a href="https://colab.research.google.com/github/larissavaladao/py6s_harmonize_sample/blob/main/6.%20parameters_by_period%20/wind_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-curuai2')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
floodplain = ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea')
floodplain

# Import ERA5 - wind - dataset

ERA5-Land is a reanalysis dataset providing a consistent view of the evolution of land variables over several decades at an enhanced resolution compared to ERA5. ERA5-Land has been produced by replaying the land component of the ECMWF ERA5 climate reanalysis. Reanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. Reanalysis produces data that goes several decades back in time, providing an accurate description of the climate of the past. This dataset includes all 50 variables as available on CDS.

ERA5-Land data is available from 1950 to three months from real-time.

Please consult the ERA5-Land "Known Issues" section. In particular, note that three components of the total evapotranspiration have values swapped as follows:

variable "Evaporation from bare soil" (mars parameter code 228101 (evabs)) has the values corresponding to the "Evaporation from vegetation transpiration" (mars parameter 228103 (evavt)),
variable "Evaporation from open water surfaces excluding oceans (mars parameter code 228102 (evaow)) has the values corresponding to the "Evaporation from bare soil" (mars parameter code 228101 (evabs)),
variable "Evaporation from vegetation transpiration" (mars parameter code 228103 (evavt)) has the values corresponding to the "Evaporation from open water surfaces excluding oceans" (mars parameter code 228102 (evaow)).
The asset is a monthly aggregate of ECMWF ERA5 Land hourly assets which includes both flow and non-flow bands. Flow bands are formed by collecting the first hour's data of the following day for each day of the month and then adding them together, while the non-flow bands are created by averaging all hourly data of the month. The flow bands are labeled with the "_sum" identifier, which approach is different from the monthly data produced by Copernicus Climate Data Store, where flow bands are averaged too.

Monthly aggregates have been pre-calculated to facilitate many applications requiring easy and fast access to the data, when sub-monthly fields are not required.

Precipitation and other flow (accumulated) bands might occasionally have negative values, which doesn't make physical sense. At other times their values might be excessively high.

This problem is due to how the GRIB format saves data: it simplifies or "packs" the data into smaller, less precise numbers, which can introduce errors. These errors get worse when the data varies a lot.

Because of this, when we look at the data for a whole day to compute daily totals, sometimes the highest amount of rainfall recorded at one time can seem larger than the total rainfall measured for the entire day.

11132 meters

Eastward and Northward  component of the 10m wind. It is the horizontal speed of air moving towards the east, at a height of ten meters above the surface of the Earth, in meters per second. Care should be taken when comparing this variable with observations, because wind observations vary on small space and time scales and are affected by the local terrain, vegetation and buildings that are represented only on average in the ECMWF Integrated Forecasting System. This variable can be combined with the V component of 10m wind to give the speed and direction of the horizontal 10m wind.

In [13]:
wind_col = (ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")
            .select(['u_component_of_wind_10m','v_component_of_wind_10m']))
# display(wind_col.size().getInfo())

In [14]:
display(wind_col.first().getInfo())

{'type': 'Image',
 'bands': [{'id': 'u_component_of_wind_10m',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]},
  {'id': 'v_component_of_wind_10m',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}],
 'version': 1738908914107439,
 'id': 'ECMWF/ERA5_LAND/MONTHLY_AGGR/195002',
 'properties': {'system:time_start': -628473600000,
  'month': 2,
  'year': 1950,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': -626054400000,
  'system:asset_size': 1069844510,
  'system:index': '195002'}}

In [15]:

# //Definir o intervalo de tempo
startyear = 2000
endyear = 2024

In [16]:
# //Criar uma lista sequencial para ano
years = ee.List.sequence(startyear,endyear);

# //Criar uma lista sequencial para meses
months = ee.List([1,4,7,10])

# //Definir nossa data inicial e final
startdate = ee.Date.fromYMD(startyear,1,1);
enddate = ee.Date.fromYMD(endyear,12,31);

In [17]:
# //filterando a coleção
wind_col = wind_col.filterDate(startdate, enddate).filterBounds(floodplain)
print('qtds',wind_col.size().getInfo())

qtds 300


In [18]:
scale = wind_col.first().select('v_component_of_wind_10m').projection().nominalScale()

In [ ]:
# //Calculo da precipitação de cada periodo da série
mean_wind_period =  ee.ImageCollection.fromImages(years.map(lambda y : months.map( lambda m : wind_col.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m,ee.Number(m).add(2),'month')).sum().set({'year': y, 'month': m,'system:time_start': ee.Date.fromYMD(y, m, 1)}).set('timestamp', ee.Date.fromYMD(y, m, 1).format('yyyy-MM-dd')))).flatten() )

display(mean_wind_period.limit(10))

In [ ]:

def mean_precip(img):
  '''receives a precipitaion image returns mean value within the floodplain limits
  as a property of the input image'''


  floodplain = ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry()


  stats = img.select("precipitation").reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': floodplain,
    'scale': chirps.first().projection().nominalScale(),
    'maxPixels': 1e10
    })

  return ee.Feature(floodplain).set({'mean_precipitation':stats.get('precipitation')}).copyProperties(img)

In [ ]:

precip_acum = ee.FeatureCollection(precipitacao_acum.map(mean_precip))
precip_acum.limit(5)